In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

In [3]:
# Define paths
trainpath = '/Users/amanmaurya/Desktop/Rice Leaf Disease Image Classification/dataset/train'
testpath = '/Users/amanmaurya/Desktop/Rice Leaf Disease Image Classification/dataset/test'

# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)


Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Tungro
Number of images in the folder: 1108
Entering to the folder name: BrownSpot
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Tungro
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)


In [4]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

# Parameters for LBP
radius = 3
n_points = 8 * radius

def compute_lbp_features(image):
    if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
        gray_image = rgb2gray(image)
    elif image.ndim == 2:  # If the image is already grayscale
        gray_image = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

def extract_lbp_features(X):
    return np.array([compute_lbp_features(img) for img in X])

In [5]:
train_feature_lbp = extract_lbp_features(X_train)
test_feature_lbp = extract_lbp_features(X_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


# LBP + Random Forest Classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test):

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_lbp, y_train)
        test_pred = rf.predict(test_feature_lbp)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'LBP+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 1.00000000, F1 Score: 0.99874687, AUC: 0.99875000
Run 2 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 0.99875622, F1 Score: 0.99874999, AUC: 0.99916667
Run 3 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 1.00000000, F1 Score: 0.99937343, AUC: 0.99937500
Run 4 - Accuracy: 0.99625000, Recall: 0.99625000, Precision: 1.00000000, F1 Score: 0.99812030, AUC: 0.99812500
Run 5 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 1.00000000, F1 Score: 0.99874687, AUC: 0.99875000
Run 6 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 1.00000000, F1 Score: 0.99937343, AUC: 0.99937500
Run 7 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 1.00000000, F1 Score: 0.99874687, AUC: 0.99875000
Run 8 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 1.00000000, F1 Score: 0.99937343, AUC: 0.99937500
Run 9 - Accuracy: 0.99875000, Recall: 0.99875000, Precision: 1.00000000, F1 Score: 0.99937343, AUC: 0.99937500
R

# LBP + Decision Tree Classifier

In [6]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test):

        dt = DecisionTreeClassifier()
        dt.fit(train_feature_lbp, y_train)
        test_pred = dt.predict(test_feature_lbp)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'LBP+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.95625000, Recall: 0.95625000, Precision: 0.95695224, F1 Score: 0.95603714, AUC: 0.97083333
Run 2 - Accuracy: 0.94000000, Recall: 0.94000000, Precision: 0.93994569, F1 Score: 0.93963669, AUC: 0.96000000
Run 3 - Accuracy: 0.94875000, Recall: 0.94875000, Precision: 0.94866945, F1 Score: 0.94850790, AUC: 0.96583333
Run 4 - Accuracy: 0.94625000, Recall: 0.94625000, Precision: 0.94657377, F1 Score: 0.94599240, AUC: 0.96416667
Run 5 - Accuracy: 0.95500000, Recall: 0.95500000, Precision: 0.95502851, F1 Score: 0.95484358, AUC: 0.97000000
Run 6 - Accuracy: 0.95500000, Recall: 0.95500000, Precision: 0.95490482, F1 Score: 0.95475411, AUC: 0.97000000
Run 7 - Accuracy: 0.95250000, Recall: 0.95250000, Precision: 0.95256410, F1 Score: 0.95234094, AUC: 0.96833333
Run 8 - Accuracy: 0.94750000, Recall: 0.94750000, Precision: 0.94740468, F1 Score: 0.94725913, AUC: 0.96500000
Run 9 - Accuracy: 0.94375000, Recall: 0.94375000, Precision: 0.94378119, F1 Score: 0.94351303, AUC: 0.96250000
R

# LBP + KNN Classifier

In [7]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test):

        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(train_feature_lbp, y_train)
        test_pred = knn.predict(test_feature_lbp)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'LBP+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 2 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 3 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 4 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 5 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 6 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 7 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 8 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
Run 9 - Accuracy: 0.97750000, Recall: 0.97750000, Precision: 0.98023770, F1 Score: 0.97808823, AUC: 0.98520833
R

# LBP + SVM Classifier

In [8]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test):

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(train_feature_lbp, y_train_cat)
        test_pred = svm.predict(test_feature_lbp)
        test_pred_proba = svm.predict_proba(test_feature_lbp)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_lbp, y_train, test_feature_lbp, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'LBP+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84513125
Run 2 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84516667
Run 3 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84510208
Run 4 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84517292
Run 5 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84528542
Run 6 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84513750
Run 7 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84529375
Run 8 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84526458
Run 9 - Accuracy: 0.64500000, Recall: 0.64500000, Precision: 0.66581334, F1 Score: 0.65082617, AUC: 0.84513958
R